In [3]:
import torch
import torchvision
from tvm import relay
import tvm
import os
import numpy as np

# Load pretrained ResNet18
resnet18 = torchvision.models.resnet18(weights="IMAGENET1K_V1").eval()

# Use a dummy input: ImageNet normalization range
dummy_input = torch.rand(1, 3, 224, 224)
dummy_input = (dummy_input - torch.tensor([0.485,0.456,0.406]).view(1,3,1,1)) / \
              torch.tensor([0.229,0.224,0.225]).view(1,3,1,1)

# Trace the model
traced_resnet18 = torch.jit.trace(resnet18, dummy_input) # Removed redundant .eval()

# Convert to TVM Relay
mod_resnet18, params_resnet18 = relay.frontend.from_pytorch(
    traced_resnet18, [("input0", dummy_input.shape)]
)

# Infer types
mod_resnet18 = relay.transform.InferType()(mod_resnet18)

# Compile with TVM
target = "llvm"
artifacts_dir = "/workspaces/TVM-Prep-guide/tvm_cpp/artifacts/ResNet18"
os.makedirs(artifacts_dir, exist_ok=True)

with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod_resnet18, target=target, params=params_resnet18)

# Export artifacts
lib.export_library(os.path.join(artifacts_dir, "ResNet18_tvm.so"))
with open(os.path.join(artifacts_dir, "ResNet18_tvm.json"), "w") as f:
    f.write(lib.get_graph_json())
    
with open(os.path.join(artifacts_dir, "ResNet18_tvm.params"), "wb") as f:
    f.write(relay.save_param_dict(lib.get_params())) 

print("✅ ResNet18 exported successfully!")

✅ ResNet18 exported successfully!
